In [ ]:
# ✅ Google Colab - HuggingFace-based RBI Risk Chatbot (LangChain + RAG + FAISS)


In [ ]:
# Step 1: Installing  Required Libraries
!pip install -U langchain langchain-community sentence-transformers faiss-cpu transformers accelerate PyPDF2 gradio



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 96.1 MB/s eta 0:00:00


In [ ]:
# Step 2: Load and Process the PDFs
from PyPDF2 import PdfReader

def load_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

In [ ]:
financial_text = load_pdf("/content/financial risk.pdf")
operational_text = load_pdf("/content/operations risk.pdf")
combined_docs = financial_text + "\n" + operational_text

In [ ]:
# Step 3: Split Text into Chunks
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
texts = splitter.split_text(combined_docs)


In [ ]:
# Step 4: Embed Text with HuggingFace Embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
docsearch = FAISS.from_texts(texts, embeddings)

In [ ]:
 #Step 5: Load HuggingFace LLM Locally (flan-t5-small)
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

In [ ]:
hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-small", max_length=512)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

Device set to use cpu


In [ ]:
# Step 6: Set up Retrieval-Augmented QA Chain
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

In [ ]:
# Step 7: Ask a Question
query = "What is operational risk according to RBI?"
response = qa_chain.run(query)
print("Bot:", response)


<ipython-input-27-5faf7f6da0bc>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)
Token indices sequence length is longer than the specified maximum sequence length for this model (943 > 512). Running this sequence through the model will result in indexing errors


Bot: Operational risk is pervasive, complex and dynamic. Unlike ma rket and credit risk, RESERVE BANK OF INDIA DEPARTMENT OF BANKING OPERATIONS AND DEVELOPMENT CENTRAL OFFICE MUMBAIINDEX GUIDANCE NOTE ON OPERATIONAL RISK MANAGEMENT Subject 1 Executive Summary 2 Background 3 Organisational set-up and Key responsibilities for Operational Risk 4 Policy requirements and strategic approach 5 Identification and Assessment of Operational Risk 6 Monitoring of Operational Risk 7 Controls / Mitigation of Operational Risk 8 Independent evaluation of Operational Risk Managem ent 9 Capital allocation for Operational Risk Annex 1 Indicative role of Organisational arm of ris k management structure Annex 2 Mapping of Business Lines Annex 3 Loss Event type classification Annex 4 Advanced Measurement Methodologies PREFACE As a step towards enhancing and fine-tuning the risk mana gement practices as also to serve as a benchmark to banks, the Reserve Bank had issu ed Guidance Notes on management of credit

In [ ]:
hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", max_length=1024)


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",  # Better than "stuff" for long docs
    retriever=docsearch.as_retriever()
)


In [ ]:
query = (
    "According to the RBI’s operational risk guidelines, "
    "how should a bank structure its risk governance framework?"
)
response = qa_chain.run(query)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2066 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
# Step 8: Optional - Gradio Chat Interface
import gradio as gr

def chatbot_interface(question):
    return qa_chain.run(question)

gr.Interface(fn=chatbot_interface, inputs="text", outputs="text", title="RBI Risk Chatbot (HuggingFace)").launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://517cf8b1f7410cdb10.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
